In [10]:

import random

values_list = random.sample(notice_to_control_time_df.notice_to_control_time.tolist(), 10)
print(f'''
notice_to_control_time_df = pd.DataFrame({{'notice_to_control_time': {values_list}}})''')


notice_to_control_time_df = pd.DataFrame({'notice_to_control_time': [565204, 66933, 180636, 170767, 266339, 255253, 244320, 82204, 93897, 363529]})


In [ ]:

.sort_values(list(columns_dict.values()), ascending=[False]*len(columns_dict.values()))

In [ ]:

            # print(previous_time, elapsed_time, session_uuid, time_group, patient_id, previous_time > elapsed_time)
            if (previous_time > elapsed_time):
                display(df1[mask_series].sort_values('elapsed_time').head(4).dropna(axis='columns', how='all').T)
                display(df2.sample(min(4, df2.shape[0])).dropna(axis='columns', how='all').T)
                raise

In [ ]:

    print(f"""
    groupby_columns = {df.columns.tolist()}
    gb = df.groupby(groupby_columns)
    rows_list = []
    for ({', '.join(df.columns)}), df1 in gb:""")

In [ ]:

    
    # Investigate the presence of trunk, neck, junctional or extremity info
    for search_str in ['trunk', 'neck', 'junctional', 'extremity']:
        srs = df2.applymap(lambda x: search_str in str(x).lower(), na_action='ignore').sum()
        columns_list = srs[srs != 0].index.tolist()
        if columns_list:
            print(search_str, columns_list)

In [ ]:

required_columns_list = [cn for cn in frvrs_logs_df.columns if cn.endswith('_required_procedure')]
mask_series = False
for cn in required_columns_list: mask_series |= (frvrs_logs_df[cn].isin(['tourniquet', 'woundpack']))

In [ ]:

fn = lambda x: ('gaze' in str(x).lower())
srs = frvrs_logs_df.applymap(fn, na_action='ignore').sum()
columns_list = srs[srs != 0].index.tolist()
print(columns_list)
for cn in columns_list:
    print(cn)
    mask_series = frvrs_logs_df[cn].map(fn)
    df = frvrs_logs_df[mask_series]
    
    # Display a sample of the data frame, dropping columns with all NaN values and transposing it
    display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)

In [ ]:


# tick_locations = ax.get_xticks()
# print(tick_locations)

# tick_labels = ax.get_xticklabels()
# print(tick_labels)

In [11]:

print('\n'.join(sorted('''**/.ipynb_checkpoints/
*.ipynb_checkpoints
*/.ipynb_checkpoints
.ipynb_checkpoints
.ipynb_checkpoints*
.ipynb_checkpoints/
.ipynb_checkpoints/*
/.ipynb_checkpoints'''.split('\n'), key=lambda x: x[::-1])))

.ipynb_checkpoints/*
.ipynb_checkpoints*
.ipynb_checkpoints/
**/.ipynb_checkpoints/
.ipynb_checkpoints
*.ipynb_checkpoints
/.ipynb_checkpoints
*/.ipynb_checkpoints
